In [8]:

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [1]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [2]:

NUM_PARTITIONS = 3
   #
DB_NAME        = "my_db"
GRAPH_NAME     = "my_graph"

print("--")


--


In [3]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [4]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


--
DB ID:  Gi6YBuTAGpEeZYvCjPfWWABD3seX7d1xMUW49KNkpzVz      DB Name:  default


In [5]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


8fZzzqacr3ywxD8RmdSy8JqwFX5Dz1CNMoyBCTXuqNBg


In [6]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, DmGQBVp5TUUjdB4bWCUps8VbuYZuMrHUmumAfSWEuwMz, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


In [ ]:
my_client.widgets().operations()


In [ ]:
# ##################################################################

In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


# Create: Vertices/nodes, edges ..

In [ ]:
import pandas as pd

print("--")

In [ ]:
#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

df_airports.head(20)


In [ ]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

df_flights.head(30)


In [ ]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

print("--")


In [ ]:
# ##################################################################

In [ ]:
#  What's in the vertices/nodes ..

l_result = my_graph.query("""

   MATCH ( n ) 
   RETURN n
   
   """)

print(l_result[0:30])


In [ ]:
#  What's in the edge .. 

l_result = my_graph.query("""

   MATCH ( n )  - [r] ->  (m)
   RETURN * //  n.id AS FROM, m.id AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS
   
   """)

print(l_result[0:30])


In [ ]:

# ##################################################################
# ##################################################################
# ##################################################################
# ##################################################################



#  Below is how I first loaded .. very slow ..


In [ ]:
#  Create nodes, airports


import contextlib

for l_each in df_airports.iterrows():
    
   l_airport_code = l_each[1][0]
   l_airport_name = l_each[1][1]
   l_label        = l_each[1][2]
    
   l_query = """

      //  This creates data that works ..
      //
      //  Subsequent execution creates duplicate nodes  :(
      //
      CREATE ( n: {2} {{ airport_code: '{0}' }} )                  //  Notice double curly braces
      SET n.airport_name = '{1}' 
      
      //  This produces bad data, see next cell ..
      //
      // MERGE ( n: {2} {{ airport_code: '{0}' }})
      // ON CREATE SET n.airport_name = '{1}'
   
      """.format(l_airport_code, l_airport_name, l_label)
         
    
   #  Don't use, hangs kernel
   #
   # with open('/dev/null', 'w') as f:
   #    with contextlib.redirect_stdout(f):
   #       l_result0 = my_graph.query(l_query)

   #  Didn't work ..
   #
   # with contextlib.redirect_stdout(None):
   #    l_result0 = my_graph.query(l_query)

   l_result0 = my_graph.query(l_query)
        
   print(".", end='')


print("  ")
print("--")


In [ ]:

for l_each in df_flights.iterrows():
    
   l_startid    = l_each[1][0]
   l_endid      = l_each[1][1]
   l_distance   = l_each[1][2]
   l_num_hops   = l_each[1][3]
   l_type       = l_each[1][4]
    
   l_query = """
      
      MATCH
         (n: Airport),
         (m: Airport)
      WHERE n.airport_code = '{0}'
      AND   m.airport_code = '{1}'
      CREATE (n) -[r: {4} {{ DISTANCE: {2}, NUM_HOPS: {3} }}]-> (m)
      
      // MATCH
      //    (n: Airport),
      //    (m: Airport)
      // WHERE n.airport_code = '{0}'
      // AND   m.airport_code = '{1}'
      // CREATE (n) -[r: {4} {{ DISTANCE: {2}, NUM_HOPS: {3} }}]-> (m)
      // RETURN type(r)
      //
      //  Using edge variables after CREATE/UPDATE/DELETE/REMOVE is not supported

      // MATCH (n: Airport ) WHERE n.airport_code = '{0}'
      // MATCH (m: Airport ) WHERE m.airport_code = '{1}'
      // CREATE (n) -[r: {4} {{ DISTANCE: {2}, NUM_HOPS: {3} }}]-> (m)

      """.format(l_startid, l_endid, l_distance, l_num_hops, l_type)
    
   l_result0 = my_graph.query(l_query)
        
   print(".", end='')     
      

print("  ")
print("--")


In [ ]:

#  Delete all edges, nodes for test reset

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   RETURN count(r)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n)
   RETURN count(n)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   DELETE r
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n)
   DELETE n
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n) - [r] -> (m)
   RETURN count(r)
   """)
display(print(l_result))

l_result = my_graph.query("""
   MATCH (n)
   RETURN count(n)
   """)
display(print(l_result))
